# Download File

In [ ]:
import requests

url = "https://www.berlin.de/sen/uvk/_assets/verkehr/verkehrsplanung/radverkehr/weitere-radinfrastruktur/zaehlstellen-und-fahrradbarometer/gesamtdatei-stundenwerte.xlsx"
response = requests.get(url)

with open("cycling_berlin_data.xlsx", "wb") as file:
    file.write(response.content)

In [31]:
import pandas as pd

xlsx = pd.ExcelFile("cycling_berlin_data.xlsx")
df_locations = xlsx.parse("Standortdaten")

columns = {
    'Zählstelle':'id',
    'Beschreibung - Fahrtrichtung': 'name',
    'Breitengrad': 'latitude',
    'Längengrad': 'longitude',
    'Installationsdatum': 'installation_date'
}

df_locations = df_locations.rename(columns=columns)

df_locations.head()

,id,name,latitude,longitude,installation_date
0,12-PA-SCH,Schwedter Steg,52.549071,13.400367,2012-03-01
1,02-MI-JAN-N,Jannowitzbrücke Nord,52.513932,13.417835,2015-04-01
2,02-MI-JAN-S,Jannowitzbrücke Süd,52.513943,13.417611,2015-04-01
3,13-CW-PRI,Prinzregentenstraße,52.488136,13.333120,2015-04-01
4,18-TS-YOR-O,Yorckstraße Ost,52.491938,13.373472,2015-04-01


In [32]:
import folium 
map = folium.Map(location=[52.52437, 13.41053], zoom_start=12)

for _, row in df_locations.iterrows():
    coordinates = [row['latitude'], row['longitude']]
    name = row['name'] + " - " + row['id']
    folium.Marker(location=coordinates, popup='Point', tooltip=name).add_to(map)

map